# Feature extraction: extraction of delta and theta frequency bands

Import packages

In [1]:
import io
import os
import mne
import copy
import glob
import array
import matplotlib
import numpy as np
import pandas as pd
import sklearn.metrics
import seaborn as sns
import scipy.io as sio
import plotly.express as px
import matplotlib.pyplot as plt


from itertools import chain

from scipy.io import loadmat
from scipy import stats

from sklearn import set_config
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, RepeatedStratifiedKFold
from sklearn.utils import resample

from autoreject import AutoReject

from collections import Counter

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import Xdawn
from mne.decoding import Vectorizer
from mne.decoding import UnsupervisedSpatialFilter

from sklearn.ensemble import BaggingClassifier


# parameters for plotting
plt.rcParams["figure.figsize"] = (10,7)

import seaborn as sns
sns.set_theme(style="whitegrid", palette="deep")

Constatnts

In [2]:
random_state = 42
signal_frequency = 256

## Load EEG data

In [4]:
def create_df_data(
    dir_path,
    info_filename=None,
    info="all",
    personal=True,
    task = 'FLA',
    reject_by_annotation=True
):
    """Loads data for all participants and create DataFrame with optional additional info from given .csv file.

    On default, loads a train set: chooses only 80% of participants
    and for each of them chooses 80% of epochs.
    It will choose them deterministically.

    Participants with less than 10 epochs per condition are rejected.

    If test_participants is set to True, it will load remaining 20% of participants.
    If test_epochs is set to True, it will load remaining 20% of epochs.
    Test epochs are chronologically after train epochs,
    because it reflects real usage (first callibration and then classification).

    Parameters
    ----------
    test_participants: bool
        whether load data for training or final testing.
        If true load participants data for testing.
    test_epochs: bool
        whether load data for training or final testing.
        If true load epochs of each participants data for testing.
    info_filename: String | None
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters
    personal: bool
        whether a model will be both trained and tested on epochs from one person
        if false, person's epochs aren't split into test and train


    Returns
    -------
    go_nogo_data_df : pandas.DataFrame

    """
    header_files_glob = os.path.join(os.path.abspath(""), dir_path + "*.vhdr")
    header_files = glob.glob(header_files_glob)

    header_files = sorted(header_files)
    go_nogo_data_df = pd.DataFrame()

    for file in header_files:
        #  load eeg data for given participant
        
        try:
            participant_epochs = load_epochs_from_file(file, task=task, reject_by_annotation=reject_by_annotation)

            # and compute participant's id from file_name
            participant_id = re.match(r".*-(\d+)_.*", file).group(1)

            f_bad = participant_epochs["f_bad"].get_data()
            f_good = participant_epochs["f_good"].get_data()

            # exclude those participants who have too few samples
            if len(f_bad) < 5 or len(f_good) < 5:
                # not enough data for this participant
                continue

            # construct dataframe for participant with: id|epoch_data|response_type|additional info...
            participant_df = create_df_from_epochs(
                participant_id, participant_epochs, info_filename, info
            )
            print(participant_id)
            go_nogo_data_df = go_nogo_data_df.append(participant_df, ignore_index=True)
        
        except:
            print("No matching events found for f_good (event id 0)")

    return go_nogo_data_df

In [5]:
def create_df_from_epochs(id, participant_epochs, info_filename, info):
    """Create df for each participant. DF structure is like: {id: String ; epoch: epoch_data ; marker: 1.0|0.0}
    1.0 means correct and 0.0 means error response.
    Default info extracted form .csv file is 'Rumination Full Scale' and participants' ids.
    With this info df structure is like:
    {id: String ; epoch: epoch_data ; marker: 1.0|0.0 ; File: id ; 'Rumination Full Scale': int}

    Parameters
    ----------
    id: String
        participant's id extracted from filename
    correct: array
        correct responses' data
    error: array
        error responses' data
    info_filename: String
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters

    Returns
    -------
    participant_df : pandas.DataFrame

    """
    participant_df = pd.DataFrame()
    info_df = pd.DataFrame()

    # get additional info from file
    if info_filename is not None:
        if info == "all":
            rumination_df = pd.read_csv(info_filename, dtype={'Demo_kod': object})
        else:
            rumination_df = pd.read_csv(info_filename, usecols=["Demo_kod"] + info)
        info_df = (
            rumination_df.loc[rumination_df["Demo_kod"] == id]
            .reset_index()
            .drop("index", axis=1)
        )
        
    epoch_df = pd.DataFrame({"id": [id], "epochs": [participant_epochs]}).join(
            info_df
        )
    participant_df = participant_df.append(epoch_df, ignore_index=True)

    return participant_df

In [6]:
def get_merged_events(raw, task):
    merged_events = None
    merged_event_dict = None
    
    if task == 'MID':
        event_dict = {
            'Stimulus/F_BAD____*ex*incentive*FB': 10001,
            'Stimulus/F_GOOD___*ex*incentive*FG': 10002,
        }

        # Map for merged correct/error response markers
        merged_event_dict = {"f_good": 0, "f_bad": 1}

        # Reconstruct the original events from Raw object
        events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

        # Merge correct/error response events
        merged_events = mne.merge_events(
            events,
            [10002],
            merged_event_dict["f_good"],
            replace_events=True,
        )

        merged_events = mne.merge_events(
            merged_events,
            [10001],
            merged_event_dict["f_bad"],
            replace_events=True,
        )
    elif task == 'GNG':
        event_dict = {
            'Stimulus/FB*ex*1_n*1_c_1*R': 10001,
            'Stimulus/FB*ex*2_n*2_c_1*R': 10002,
            'Stimulus/FG*ex*1_n*1_c_1*R': 10003,
            'Stimulus/FG*ex*2_n*2_c_1*R': 10004,
        }

        # Map for merged correct/error response markers
        merged_event_dict = {"f_good": 0, "f_bad": 1}

        # Reconstruct the original events from Raw object
        events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

        # Merge correct/error response events
        merged_events = mne.merge_events(
            events,
            [10003, 10004],
            merged_event_dict["f_good"],
            replace_events=True,
        )

        merged_events = mne.merge_events(
            merged_events,
            [10001, 10002],
            merged_event_dict["f_bad"],
            replace_events=True,
        )
        
    elif task == 'FLA':
        event_dict = {
            'Stimulus/F_BAD___*ex**lll*l*FB': 10001,
            'Stimulus/F_BAD___*ex**lrl*r*FB': 10002,
            'Stimulus/F_BAD___*ex**rlr*l*FB': 10003,
            'Stimulus/F_BAD___*ex**rrr*r*FB': 10004,
            'Stimulus/F_GOOD__*ex**lll*l*FG': 10005,
            'Stimulus/F_GOOD__*ex**lrl*r*FG': 10006,
            'Stimulus/F_GOOD__*ex**rlr*l*FG': 10007,
            'Stimulus/F_GOOD__*ex**rrr*r*FG': 10008,
        }

        # Map for merged correct/error response markers
        merged_event_dict = {"f_good": 0, "f_bad": 1}

        # Reconstruct the original events from Raw object
        events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

        # Merge correct/error response events
        merged_events = mne.merge_events(
            events,
            [10005, 10006, 10007, 10008],
            merged_event_dict["f_good"],
            replace_events=True,
        )

        merged_events = mne.merge_events(
            merged_events,
            [10001, 10002, 10003, 10004],
            merged_event_dict["f_bad"],
            replace_events=True,
        )
    return merged_events, merged_event_dict      

In [7]:
def load_epochs_from_file(file, task, reject_by_annotation=True, mask=None):
    """Load epochs from a header file.

    Args:
        file: path to a header file (.vhdr)
        reject_bad_segments: 'auto' means that bad segments are rejected automatically.

    Returns:
        mne Epochs

    """
    # Import the BrainVision data into an MNE Raw object
    raw = mne.io.read_raw_brainvision(file, eog=('HEOG', 'VEOG'))

    # Construct annotation filename
    annot_file = file[:-4] + "vmrk"

    # Read in the event information as MNE annotations
    annotations = mne.read_annotations(annot_file)

    # Add the annotations to our raw object so we can use them with the data
    raw.set_annotations(annotations)

    # Map with response markers only GNG
    merged_events, merged_event_dict = get_merged_events(raw, task)
    
    tmin=None
    tmax=None
    
    if task == 'FLA' or task == 'GNG':
        tmin, tmax = -1.0, 2.4  # Start and end of the segments
    elif task == 'MID':
        tmin, tmax = -0.2, 0.8  # Start and end of the segments

# Read epochs
    epochs = mne.Epochs(
        raw=raw,
        events=merged_events,
        event_id=merged_event_dict,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        reject_by_annotation=reject_by_annotation,
        preload=True,
        verbose=0,
    )

    if not reject_by_annotation:
        ar = AutoReject(random_state=random_state, n_jobs=10, verbose=0)
        epochs_ar, reject_log = ar.fit_transform(epochs, return_log=True)
        epochs = epochs_ar        
    
    return epochs

Choose the data to load

In [8]:
load_autoreject_data = True
task = 'GNG'

In [ ]:
# %%capture
info_filename = 'data/sonata_data/questionnaires.csv'

if load_autoreject_data:
    df_name = f'sonata_data_{task}_autoreject'
    dir_name = f'data/sonata_data/feedback_export_{task}/'
    pickled_data_filename = 'data/sonata_data/' + df_name + '.pkl'

    if os.path.isfile(pickled_data_filename):
        print("Pickled file found. Loading pickled data...")
        data_df = pd.read_pickle(pickled_data_filename)
        data_df.name = df_name
        print("Done")
        pass
    else:
        print("Pickled file not found. Loading data...")
        data_df = create_df_data(
            dir_path=dir_name, 
            info="all", 
            personal=False, 
            info_filename=info_filename, 
            task=task,
            reject_by_annotation = False
        )
        
        data_df.name = df_name
        # save loaded data into a pickle file
        data_df.to_pickle("data/sonata_data/" + data_df.name + ".pkl")
        print("Done. Pickle file created")
else:
    
    df_name = f'sonata_data_{task}'
    dir_name = f'data/sonata_data/feedback_export_{task}/'
    pickled_data_filename = 'data/sonata_data/' + df_name + '.pkl'

    if os.path.isfile(pickled_data_filename):
        print("Pickled file found. Loading pickled data...")
        data_df = pd.read_pickle(pickled_data_filename)
        data_df.name = df_name
        print("Done")
        pass
    else:
        print("Pickled file not found. Loading data...")
        data_df = create_df_data(
            dir_path=dir_name, 
            info="all", 
            personal=False, 
            info_filename=info_filename, 
            task=task,
            reject_by_annotation = True
        )
        data_df.name = df_name
        # save loaded data into a pickle file
        data_df.to_pickle("data/sonata_data/" + data_df.name + ".pkl")
        print("Done. Pickle file created")

### Add frequency data: delta and theta

In [19]:
freq_ranges = [(0.1,3.0), (4.0,7.5)]

- add delta

In [20]:
%%capture

data_df['delta'] = data_df['epochs'].apply(lambda x: x.copy().filter(
    None, 
    freq_ranges[0][1], 
    fir_design='firwin', 
    skip_by_annotation='edge',
    h_trans_bandwidth = 'auto',
))

- add theta

In [22]:
%%capture

data_df['theta'] = data_df['epochs'].apply(lambda x: x.copy().filter(
    freq_ranges[1][0], 
    freq_ranges[1][1], 
    fir_design='firwin', 
    skip_by_annotation='edge',
    h_trans_bandwidth = 'auto',
))

Save the results

In [24]:
data_df.to_pickle("data/sonata_data/" + data_df.name + "_freq.pkl")

## Define experimental and control groups

Read data with frequencies

In [11]:
data_df = pd.read_pickle('data/sonata_data/sonata_data_GNG_autoreject_freq_short.pkl')

In [ ]:
dep = data_df[(data_df['BDI'] > 13) & (data_df['STAI'] > 41)]
ctrl_dep = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] > 41)]
anx = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] > 42)]
ctrl_anx = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] < 41)]

#### Summary of groups stats

In [121]:
def df_add_demo_info(demo_file_name, df):
    demo_df = pd.read_csv(f'data/sonata_data/{demo_file_name}', dtype={'Demo_kod': object}, usecols=[
        'Demo_kod',
        'Płeć', 
        'Ręczność',
        'Wiek',
        'Wzrost',
        'Waga',
        'Twoja dotychczasowa liczba lat edukacji (w pełnych latach)'
        ]
    )
        
    demo_df.set_index('Demo_kod', inplace=True)
    
    group_indexes = df['Demo_kod'].to_list()
    demo_ = demo_df.loc[group_indexes]
        
    df_indexed = df.set_index('Demo_kod')
    
    df_demo = pd.concat([df_indexed,demo_], axis=1)
        
    df_demo.rename(columns = {
        'Płeć':'Sex',
        'Wiek': 'Age',
        'Ręczność': 'Handedness',
        'Wzrost': 'Height',
        'Waga': 'Weight',
        'Twoja dotychczasowa liczba lat edukacji (w pełnych latach)':'Education (years)'
    }, inplace=True)
    
    df_demo['Sex'] = df_demo['Sex'].astype(float)
    
    return df_demo

1. Full sample

In [138]:
demo_df = pd.read_csv(f'data/sonata_data/Sonata_scales.csv', dtype={'Demo_kod': object}, usecols=[
        'Demo_kod',
        'Płeć', 
        'Ręczność',
        'Wiek',
        'Wzrost',
        'Waga',
        'Twoja dotychczasowa liczba lat edukacji (w pełnych latach)'
        ]
)

demo_df.describe()

,Płeć,Ręczność,Wiek,Wzrost,Waga,Twoja dotychczasowa liczba lat edukacji (w pełnych latach)
count,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000
mean,0.495556,0.897778,23.640000,173.153333,68.186222,15.413333
std,0.499980,0.303616,4.176807,9.633220,14.820595,2.400298
min,0.000000,0.000000,18.000000,152.000000,20.000000,9.000000
25%,0.000000,1.000000,21.000000,165.000000,58.000000,14.000000
50%,0.000000,1.000000,23.000000,173.000000,66.000000,15.000000
75%,1.000000,1.000000,25.000000,180.000000,76.000000,17.000000
max,1.000000,1.000000,39.000000,200.000000,142.000000,25.000000


Gender info

In [140]:
sex = demo_df['Płeć'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys([0.0, 1.0, 0.5])
dict_values([113, 111, 1])


2. Sample after artifact rejection

In [143]:
data_full_demo_df = df_add_demo_info(demo_file_name = 'Sonata_scales.csv', df=data_df)
data_full_demo_df.describe()

,BDI,BDI_cognitive,BDI_affective,BDI_affect_cog,BDI_somatic,STAI,Sex,Handedness,Age,Height,Weight,Education (years)
count,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000
mean,11.959459,0.514324,0.499324,1.014550,0.611171,45.567568,0.493243,0.896396,23.558559,173.186937,68.062613,15.409910
std,10.231845,0.549190,0.544566,0.995998,0.503103,9.136097,0.499954,0.305434,4.096538,9.569345,14.759695,2.408223
min,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,18.000000,152.000000,20.000000,9.000000
25%,5.000000,0.170000,0.000000,0.210000,0.170000,39.000000,0.000000,1.000000,21.000000,165.250000,57.250000,14.000000
50%,9.000000,0.330000,0.330000,0.670000,0.500000,45.000000,0.000000,1.000000,23.000000,173.000000,66.000000,15.000000
75%,16.750000,0.830000,0.670000,1.500000,0.830000,52.000000,1.000000,1.000000,25.000000,180.000000,76.000000,17.000000
max,48.000000,2.830000,2.330000,5.170000,2.330000,70.000000,1.000000,1.000000,39.000000,200.000000,142.000000,25.000000


Gender info

In [144]:
sex = data_full_demo_df['Sex'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys([0.0, 1.0, 0.5])
dict_values([112, 109, 1])


3. Groups

- Depression

In [119]:
dep_demo = df_add_demo_info(demo_file_name = 'Sonata_scales.csv', df=dep)
dep_demo.describe()

,BDI,BDI_cognitive,BDI_affective,BDI_affect_cog,BDI_somatic,STAI,Sex,Handedness,Age,Height,Weight,Education (years)
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,23.466667,1.053333,0.977067,2.031067,1.093467,54.106667,0.413333,0.893333,23.080000,171.133333,64.980000,15.466667
std,8.855191,0.570844,0.588644,0.993065,0.474545,6.235412,0.495748,0.310768,3.451439,9.719489,12.538169,2.372895
min,14.000000,0.000000,0.000000,0.670000,0.330000,42.000000,0.000000,0.000000,18.000000,155.000000,45.000000,11.000000
25%,16.000000,0.670000,0.330000,1.330000,0.830000,50.000000,0.000000,1.000000,21.000000,163.500000,55.000000,14.000000
50%,21.000000,1.000000,1.000000,1.830000,1.000000,55.000000,0.000000,1.000000,23.000000,169.000000,64.000000,15.000000
75%,28.500000,1.170000,1.330000,2.500000,1.330000,58.000000,1.000000,1.000000,25.000000,177.500000,71.000000,17.000000
max,48.000000,2.830000,2.330000,5.170000,2.330000,70.000000,1.000000,1.000000,38.000000,198.000000,97.000000,25.000000


In [128]:
sex = dep_demo['Sex'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys([1.0, 0.0])
dict_values([31, 44])


- Control for depression

In [122]:
dep_ctrl_demo = df_add_demo_info(demo_file_name = 'Sonata_scales.csv', df=ctrl_dep)
dep_ctrl_demo.describe()

,BDI,BDI_cognitive,BDI_affective,BDI_affect_cog,BDI_somatic,STAI,Sex,Handedness,Age,Height,Weight,Education (years)
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.00000,72.000000,72.000000,72.000000,72.000000
mean,7.069444,0.310139,0.300278,0.611667,0.389028,47.194444,0.437500,0.87500,23.361111,172.381944,67.402778,15.055556
std,3.678358,0.271451,0.318044,0.426902,0.294351,3.550992,0.496023,0.33304,4.280008,8.794802,17.550458,2.075266
min,0.000000,0.000000,0.000000,0.000000,0.000000,42.000000,0.000000,0.00000,18.000000,155.000000,20.000000,9.000000
25%,4.000000,0.000000,0.000000,0.290000,0.170000,45.000000,0.000000,1.00000,20.000000,165.000000,56.750000,14.000000
50%,7.000000,0.330000,0.330000,0.670000,0.330000,47.000000,0.000000,1.00000,22.000000,172.500000,63.500000,15.000000
75%,11.000000,0.500000,0.330000,0.830000,0.500000,50.000000,1.000000,1.00000,25.000000,179.250000,76.500000,16.250000
max,13.000000,1.170000,1.670000,1.830000,1.500000,55.000000,1.000000,1.00000,39.000000,190.000000,142.000000,20.000000


In [129]:
sex = dep_ctrl_demo['Sex'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys([0.0, 1.0, 0.5])
dict_values([40, 31, 1])


- Anxiety

In [96]:
anx_demo = df_add_demo_info(demo_file_name = 'Sonata_scales.csv', df=anx)
anx_demo.describe()

,BDI,BDI_cognitive,BDI_affective,BDI_affect_cog,BDI_somatic,STAI,Handedness,Age,Height,Weight,Education (years)
count,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000
mean,7.015385,0.310308,0.301846,0.613231,0.377231,47.753846,0.861538,23.430769,172.576923,67.615385,15.046154
std,3.676328,0.263106,0.322005,0.439961,0.293260,3.274097,0.348072,4.264568,8.700409,18.050011,2.139150
min,0.000000,0.000000,0.000000,0.000000,0.000000,43.000000,0.000000,18.000000,157.000000,20.000000,9.000000
25%,4.000000,0.170000,0.000000,0.170000,0.170000,45.000000,1.000000,20.000000,165.000000,56.000000,14.000000
50%,7.000000,0.330000,0.330000,0.670000,0.330000,47.000000,1.000000,22.000000,173.000000,64.000000,15.000000
75%,10.000000,0.500000,0.330000,0.830000,0.500000,50.000000,1.000000,25.000000,180.000000,78.000000,17.000000
max,13.000000,1.170000,1.670000,1.830000,1.500000,55.000000,1.000000,39.000000,190.000000,142.000000,20.000000


In [130]:
sex = anx_demo['Sex'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys(['0', '1', 'Osoba niebinarna'])
dict_values([36, 28, 1])


- Control for anxiety

In [131]:
anx_ctrl_demo = df_add_demo_info(demo_file_name = 'Sonata_scales.csv', df=ctrl_anx)
anx_ctrl_demo.describe()

,BDI,BDI_cognitive,BDI_affective,BDI_affect_cog,BDI_somatic,STAI,Sex,Handedness,Age,Height,Weight,Education (years)
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,4.439394,0.151818,0.181061,0.333788,0.305455,34.893939,0.636364,0.909091,24.393939,176.090909,71.833333,15.878788
std,3.314902,0.203567,0.262433,0.394367,0.274351,4.210349,0.484732,0.289683,4.643867,9.530078,12.981150,2.792931
min,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,19.000000,152.000000,45.000000,12.000000
25%,1.250000,0.000000,0.000000,0.000000,0.170000,32.000000,0.000000,1.000000,21.000000,170.000000,62.000000,14.000000
50%,4.500000,0.170000,0.000000,0.170000,0.330000,35.000000,1.000000,1.000000,23.000000,176.000000,71.500000,15.000000
75%,6.750000,0.170000,0.330000,0.627500,0.500000,38.750000,1.000000,1.000000,26.000000,182.750000,80.750000,17.000000
max,12.000000,1.000000,1.330000,1.500000,1.330000,40.000000,1.000000,1.000000,39.000000,200.000000,107.000000,24.000000


In [132]:
sex = anx_ctrl_demo['Sex'].to_list()

print(Counter(sex).keys()) # equals to list(set(words))
print(Counter(sex).values()) # counts the elements' frequency

dict_keys([0.0, 1.0])
dict_values([24, 42])


### Test differences in BDI and STAI scores between groups

#### Depression

- STAI

In [78]:
control_depression_tai = ctrl_dep['STAI'].to_numpy()
depression_tai = dep['STAI'].to_numpy()

print(f"Depression group: mean TAI score: {depression_tai.mean()} SD = {depression_tai.std()}")
print(f"Control group: mean TAI score: {control_depression_tai.mean()} SD = {control_depression_tai.std()}")

t_value, p_value = ttest_rel(depression_tai[1:], control_depression_tai)
print(f"t({len(control_depression_tai) -1}) = {t_value}, p = {p_value}")

- BDI

In [83]:
control_depression_bdi = ctrl_dep['BDI'].to_numpy()
depression_bdi = dep['BDI'].to_numpy()

print(f"Depression group: mean BDI score: {depression_bdi.mean()} SD = {depression_bdi.std()}")
print(f"Control group: mean BDI score: {control_depression_bdi.mean()} SD = {control_depression_bdi.std()}")

t_value, p_value = ttest_rel(depression_bdi[1:], control_depression_bdi)
print(f"t({len(depression_bdi[1:]) -1}) = {t_value}, p = {p_value}")

#### Anxiety

- STAI

In [88]:
control_anx_tai = ctrl_anx['STAI'].to_numpy()
anx_tai = anx['STAI'].to_numpy()

print(f"Anxiety group: mean TAI score: {anx_tai.mean()} SD = {anx_tai.std()}")
print(f"Control group: mean TAI score: {control_anx_tai.mean()} SD = {control_anx_tai.std()}")

t_value, p_value = ttest_rel(anx_tai, control_anx_tai[2:])
print(f"t({len(anx_tai[1:]) -1}) = {t_value}, p = {p_value}")

- BDI

In [93]:
control_anx_bdi = ctrl_anx['BDI'].to_numpy()
anx_bdi = anx['BDI'].to_numpy()

print(f"Anxiety group: mean BDI score: {anx_bdi.mean()} SD = {anx_bdi.std()}")
print(f"Control group: mean BDI score: {control_anx_bdi.mean()} SD = {control_anx_bdi.std()}")

t_value, p_value = ttest_rel(anx_bdi[1:], control_anx_bdi)
print(f"t({len(anx_bdi[1:]) -1}) = {t_value}, p = {p_value}")